In [3]:
from pyMON import pyMON
import lightkurve as lk 
%matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

Using matplotlib backend: <object object at 0x78a1585305e0>


In [4]:
## Load KEPSEISMIC lightcurve for KIC 2707716

Star_ID = 2707716

filename_lc = f'./example_files/hlsp_kepseismic_kepler_phot_kplr00{Star_ID}-55d_kepler_v1_cor-filt-inp.fits'

hdul_lc = fits.open(filename_lc)

star_lc = Table(hdul_lc[1].data).to_pandas()

hdul_lc.close()

time = star_lc.TIME.values

### Load KEPSEISMIC power spectrum for KIC 2707716

filename_psd = f'./example_files/hlsp_kepseismic_kepler_phot_kplr00{Star_ID}-55d_kepler_v1_cor-psd-filt-inp.fits'

hdul_psd = fits.open(filename_psd)

star_psd = Table(hdul_psd[1].data).to_pandas()

hdul_psd.close()

frequency, power = star_psd.TIME.values, star_psd.FLUX.values

frequency = frequency * 10**6   # convert to muHz

In [5]:
## Run pyMON

# Smoothing scaling parameter
sm = 2

# Lower frequency to search for power excess. If set to None, the lowerp frequency will be determined from a full width half max estimate
lowerp = None

# Upper frequency to search for power excess. If set to None, the upperp frequency will be determined from a full width half max estimate
upperp = None

# background model to fit to power spectrum. Choice between: ['nuSYD', 'harvey', 'linear', 'white']
bkg_model = 'linear'

# an estimate for numax in muHz
numax_est = 30.42  # from APOKASC3 (Pinsonneault et al., 2025)

# number of interactions for the mc uncertainty sampling routine. Default is 500. Set to '0' to not estimate uncertainties
mc_iters = 500

# either a key which points to a coefficient and exponent in a seperate file, or a coefficient+exponent in a list
# for the a general dnu-numax relation derived from 16,000 Kepler stars, use the key 'Yu18' (Yu et al., 2018) 
Dnu_relation = 'Yu18'


# save inputs as a dictionary (must be passed into pyMON as a dict with all parameters)
inputs = {'sm': sm, 'lowerp': lowerp, 'upperp': upperp, 'background_model': bkg_model, 
          'numax_est': numax_est, 'mc_iters': mc_iters, 'Dnu_relation': Dnu_relation}



pyMON_df = pyMON(frequency, power, time, Star_ID, inputs)



pyMON_df

Parameters for 2707716:
White noise metric=  383.94696775033657  ppm^2/muHz
Measured numax =  30.09253865656354
Measured Amplitude =  9141.914534045487
Measured width =  14.044652277167188
------------------------------------------------------
Running sampling routine:


100%|████████████████████████████████████████▉| 499/500 [00:15<00:00, 32.09it/s]

Parameters for 2707716:
Nyquist frequency (from light curve) = 283.2099728978449 muHz
Measured numax =  30.09253865656354 +/- 0.7180218146299191 muHz ( 2.386045999058009 %)
Measured Amplitude =  9141.914534045487 +/- 898.717043007104 ppm^2/muHz
Measured width =  14.044652277167188 +/- 0.930164623497853 muHz


,Star_ID,numax_predicted,Dnu_relation,lowerp,upperp,sm,background_model,numax_measured,numax_uncertainty,peak_measured,peak_uncertainty,width_measured,width_uncertainty,mc_iters
0,2707716,30.543817,Yu18,None,None,2,linear,30.092539,0.718022,9141.914534,898.717043,14.044652,0.930165,500


100%|█████████████████████████████████████████| 500/500 [00:30<00:00, 32.09it/s]